# Gas Diffusion Modeling

## Negatrode:
I am going to break down the 1 mm negatrode into two parts: a gas diffusion part, and a reaction volume.  The first 980 $\mathrm{\mu}$m of the negatrode are too far away from the electrolyte for the reactions to really mattter so this will just be for gas diffusion.  In the last 20 $\mathrm{\mu}$m there will be gas diffusion as well as reactions with the negatrode.  The microstructure is the same in both sections.

For the purposes of this model I will be running in fuel cell mode and the gas composition will look like this:
 - Negatrode side 50% H2 50% N2
 - Positive Side: 10% Steam, 19% $O_2$, and 71% $N_2$.
 
 
I am assuming that there is enough flow that the layers closest to the inlet gasses will always be at the concentrations of the inflowing gas.  The positrode is only a 20 micron thick layer, so the gas will always be at the concentration of the inflow channels, so for now I will not model the gas flow in the positrode

### Negatrode
Node 1: in the first 980 microns of the negatrode, I am assuming that there is enough flow for the gas to stay at the same pressure/composition. This node is at the boundary of the reaction volume, so some hydrogen has been converted to water
 - Pressure = 101,325 Pa
 - dy = 980 microns
 - $X_{\rm H_2} = 0.50$
 - $X_{\rm N_2} = 0.50$
 - $X_{\rm H_2O} = 0.02$


Node 2: in the reaction layer, where H$_2$ is consumed and H$_2$O is produced throught the reaction volume. I  am also just guessing the gas composition here.
 - Pressure = 81,343 Pa (will likely be at a lower pressure than the top layer of the negatrode, this is not an actual calculated value I just used the atmosphereic pressure in golden as a number lower than atmospheric pressure). 
 - dy = 20 microns
 - $X_{\rm H_2} = 0.40$
 - $X_{\rm N_2} = 0.55$
 - $X_{\rm H_2O} = 0.05$
 
The gas-phase species diffusion coefficients are: Using your numbers from HW5
- N$_2$: $2.798\times 10^{-5}\,\frac{\rm m^2}{\rm s}$
- H$_2$O: $1.9\times 10^{-5}\,\frac{\rm m^2}{\rm s}$
- H$_2$: $0.3\times 10^{-5}\,\frac{\rm m^2}{\rm s}$ Reasonable looking number

### Assumptions:
 - n is -0.5
 - That the negatrode and positrode microstructures are roughly packed spheres
 - That there is enough flow for the gas to stay at the same composition
 - The bulk diffusion of species in the negatrode will have a negligable effect on performance if they occur more than 20 microns from the electrolyte
 - Adsorption and desorption reactions in the gas diffusion volume of the negatrode are in steady state and will not effect concentrations
 - The negatrode microstructure is homogeneus throughout the volume

# New Solution Vector
There will now be additions to the solution vector, which are the species concentrations at different nodes:
 - C_k_top
 - C_k_rxn 

Although I am starting with mol fractions, I will be storing the values in the state function as the concentrations.  This is because having the concentration value will make it easier to calculate the gas adsorption/desorption rates. Also it just keeps the solution vector consistent in tracking concentrations.

The vector will now be a nested array due to there being multiple anode species. making a new SV: (well the ideal SV)
$$SV = \begin{bmatrix} \phi_\mathrm{int,neg} \\ C_\mathrm{k,top,gas} \\ C_\mathrm{k,rxn,gas} \\ C_\mathrm{H,surf,Ni} \\ C_\mathrm{H^+,surf,Ni} \\ C_{O^{2-},surf,Ni} \\ C_{H_2O,surf,Ni} \\ C_\mathrm{[],surf,Ni} \\ C_\mathrm{[],surf,elyte}\\ \phi_\mathrm{int,pos} \\ C_\mathrm{H^+,surf,BCFZY} \\ C_{O^{2-},surf,BCFZY} \\ C_{H_2O,surf,BCFZY} \\ C_\mathrm{O,surf,BCFZY} \\ C_\mathrm{[],surf,BCFZY} \end{bmatrix}$$

New actual SV:
$$SV = \begin{bmatrix} \phi_\mathrm{int,neg} \\ C_\mathrm{k,top,gas} \\ C_\mathrm{k,rxn,gas} \\ \phi_\mathrm{int,pos}  \end{bmatrix}$$

In [1]:
#Kg calculations
#----- Negatrode:
#inputs
eps_Ni = 0.159 #see calculations
eps_elyte_neg = 0.191 #See Calculations
eps_gas_neg = 1-eps_Ni-eps_elyte_neg
d_Ni_neg = 1*10**-5 #(m)rough estimate from SEM images (average diameter of Ni in negatrode)
d_elyte_neg = 5*10**-6 #(m) rough estimate from SEM images (average diameter of BCZYYb in negatrode)
n_brugg = -0.5

#T,fac:
tau_fac_neg = eps_gas_neg**n_brugg

#Kg
d_part_avg = (d_Ni_neg+d_elyte_neg)/2 #just taking a linear average of the two particle sizes
Kg_neg = (eps_gas_neg**3*d_part_avg**2)/(72*tau_fac_neg*(1-eps_gas_neg)**2)


In [2]:
# inputs:
import numpy as np
# --- Negatrode
dy1 = 980e-6 # m
dy2 = 20e-6 # m

eps_g_neg = 1-eps_Ni-eps_elyte_neg
Kg_neg = (eps_gas_neg**3*d_part_avg**2)/(72*tau_fac_neg*(1-eps_gas_neg)**2)
tau_fac_neg = eps_gas_neg**n_brugg

T = 823 # K

P_neg_gd = 101325 # Pa
P_neg_rxn = 81343 # Pa

R = 8.3145 # J/mol-K
F = 96485  # C/mol equiv

# mol fractions
X_k_gd = np.array([0.50, 0.50, 0.0]) #H2, N2, Steam
X_k_rxn = np.array([0.40, 0.55, 0.05]) #H2, N2, Steam

#Concentrations
C_k_gd_0 = X_k_gd*((P_neg_gd)/(R*T)) #H2, N2, Steam
C_k_rxn_0 = X_k_rxn*((P_neg_rxn)/(R*T)) #H2, N2, Steam

mu = 2.08e-5 #kg/m-s #I am going to use your value for this

D_k_an = np.array([0.3e-5, 2.798e-5, 1.9e-5]) #m2/s, H2, N2, Steam

In [3]:
#Dictionaries:
s1 = {'C_k':C_k_gd_0,'dy':dy1}
s2 = {'C_k':C_k_rxn_0,'dy':dy2}
gasProps = {'Kg':Kg_neg,'t_fac':tau_fac_neg,'D_k':D_k_an, 'mu':mu,'T':T}


In [4]:
def gas_diffusion(s1,s2,gasProps):
    N_k  = np.zeros_like(s1['C_k'])
    
    #Setting the volume fractions of each layer of the negatrode:
    f1 = s1['dy']/(s1['dy'] + s2['dy'])
    f2 = 1-f1
    
    C_int = f1*s1['C_k'] + f2*s2['C_k']
    
    #re-finding the mol fractions of the gas constituents
    X_k_1 = s1['C_k']/np.sum(s1['C_k'])
    X_k_2 = s2['C_k']/np.sum(s2['C_k'])
    X_k_int = f1*X_k_1 + f2*X_k_2
    
    #Calculating the pressure values
    P_1 = np.sum(s1['C_k'])*R*gasProps['T']
    P_2 = np.sum(s2['C_k'])*R*gasProps['T']
    
    #Calculating V_k_diff
    D_k_eff = eps_g_neg*gasProps['D_k']/tau_fac_neg #eps_g_neg and tau_fac_neg will be solved for before the function
    dY = 0.5*(s1['dy'] + s2['dy']) #getting the average thickness for each layer
    V_k_diff = -D_k_eff*(X_k_2 - X_k_1)/(dY*X_k_int)
    
    V_conv = -Kg_neg*(P_2 - P_1)/(dY*gasProps['mu']) #Kg_neg will be solved for before the function
    V_k_diff = -D_k_eff*(X_k_2 - X_k_1)/(dY*X_k_int)

    V_k  = V_conv + V_k_diff
    
    N_k = C_int*X_k_int*V_k
    return N_k

## Future Work:
well... assuming I have the time/motivation.

I Would like to account for transport losses. Thus, relaxing the assumption that I will be flowing enough fuel in the gass channels such that the gas composition will vary if the cell is running at high current densities.  For this I will have to model in the gas channel in the Positrode and Negatrode.  Thus making the negatrode 3 nodes and the positrode 2 nodes. Detailed below:

I would also like to run the cell in electrolysis mode, meaning that I will be flowing a sweep gas with a small concentration of hydrogen on the negatrode side and humid air on the positrode side. The total gas flow on either side will be 100 SCCM
 - Negatrode side 50% H2 50% N2
 - Positive Side: 30% Steam, 15% $O_2$, and 55% $N_2$.

### Negatrode
Node 1: Gas diffusion channel, I will assume this is about 1mm thick

Node 2: From the Gas diffusion channel to the reaction zone. This is node 1 in the first model.

Node 3: last 20 microns of the negatrode and is node 2 in the first model


### Positrode
Node 1: Gas flow channel. The initial conditions will be atmospheric pressure, and the initial gas flow concentrations. I will assume this is about 1mm thick

Node 2: From gas flow channel to the positrode/electrolyte interface, where O$_2$ is consumed and H$_2$O is produced throught the volume of the positrode. this layer will be the whole positrode and will be about 20 microns thick.